In [96]:
## Not great, has names

with open('words.txt', 'r') as f:
    allwords = f.read().split()

In [111]:
ALLCAPS = {l:u for (l,u) in zip('abcdefghijklmnopqrstuvwxyz', 'ABCDEFGHIJKLMNOPQRSTUVWXYZ')}

def chunks(w):
    return [e for e in w]

def compare(w1, w2):
    """First round of comparing: positional comparison"""
    return [a==b for (a,b) in zip(w1, w2)]

def count_c(c, w):
    return len([e for e in word if e==c])

def show_flags(guess, flags):
    """print guess along with its flags side by side"""
    for c in guess:
            print(ALLCAPS[c], '', end='')
    print('\n')
    for f in flags:
        print(f, '', end='')
    
class Wordle:
    def __init__(self, word):
        self.word = word
        self.length = len(word)
        self.history = []
        self.optimal = ['_' for i in range(self.length)]
        self.char_bank = 'abcdefghijklmnopqrstuvwxyz'
    
    def verify(self, guess):
        flags = []
        if guess == self.word:
            print('correct!')
            flags = ['🟩' for i in range(self.length)]
            return
        word2word = compare(guess, self.word)
        for ix, b in enumerate(word2word):
            if b:
                flags.append('🟩') #green
                self.optimal[ix] == guess[ix]
            else:
                ## TODO need to account for character count. e.g., if character count of 1
                ## and shown yellow flag, should be gray on its next encounter in the guess
                if guess[ix] in self.word: 
                    flags.append('🟨') #yellow
                else:
                    flags.append('⬛️') #gray
                    self.char_bank.replace(guess[ix],'')
        
        self.history.append(flags)
        show_flags(guess, flags)
        return flags

In [112]:
word = 'crimp'
x = Wordle(word)
x.verify('floni')

F L O N I 

⬛️ ⬛️ ⬛️ ⬛️ 🟨 

['⬛️', '⬛️', '⬛️', '⬛️', '\U0001f7e8']

# algorithm

1. start with n-letter word with highest entropy
2. if green, fix position
3. if yellow, switch position

Data science questions:
- are there any words for which it is very hard for this algorithm (or human) to guess within 6 tries?

In [130]:
import numpy as np

def entropy(word):
    prob_log_prob = []
    for c in set(word):
        p = count_c(c, word)/len(word)
        prob_log_prob.append(p*np.log(p))
    return -sum(prob_log_prob)

bank = [w for w in allwords if len(w) == len(word)]

start = bank[np.argmax([entropy(w) for w in bank])] # should start with random choice from all high entropy words
for t in range(6):
    